In [1]:
import pandas as pd
train=pd.read_csv('data/train_dataset_v2.tsv', sep='\t')

In [2]:
print(train['emotions'])

0        0,0,0,0,0,0
1        0,0,0,0,0,0
2        0,0,0,0,0,0
3        0,0,0,0,0,0
4        0,0,0,0,0,0
            ...     
42785    0,0,0,0,0,0
42786    0,3,0,0,0,0
42787    2,3,0,0,0,0
42788    2,3,0,0,0,0
42789    0,0,0,0,0,0
Name: emotions, Length: 42790, dtype: object


In [3]:
content_dict = {}
for index in range(len(train['content'])):
    currentindex = train['id'][index]
    currentindex = currentindex.split('_')
    str1 = currentindex[0]
    while len(str1) < 5:
        str1 = '0'+str1
    str4 = currentindex[3]
    while len(str4) < 4:
        str4 = '0'+str4
    resultindex = str1+'_'+currentindex[1]+'_'+currentindex[2]+'_'+str4
    content_dict[resultindex] = train['content'][index]

In [4]:
def sortedDictValues(adict): 
    keys = adict.keys() 
    keys = sorted(keys)
    print('keys = ')
    print(keys[0:10])
    new_content_dict = {}
    for data in keys:
        new_content_dict[data] = content_dict[data]
    return new_content_dict 
new_content_dict = sortedDictValues(content_dict)

keys = 
['01171_0001_A_0001', '01171_0001_A_0002', '01171_0001_A_0003', '01171_0001_A_0004', '01171_0001_A_0005', '01171_0001_A_0006', '01171_0001_A_0007', '01171_0001_A_0008', '01171_0001_A_0009', '01171_0001_A_0010']


In [5]:
inv_new_content_dict = {}
for data in new_content_dict:
    inv_new_content_dict[new_content_dict[data]] = data

In [6]:
keys = new_content_dict.keys()
keys = list(keys)
dict_keys_index = {}
for index in range(len(keys)):
    #dict_keys_index[index] = keys[index]
    dict_keys_index[keys[index]] = index

In [7]:
content,emotions = [],[]
dicts = {}
inv_dicts = {}
label_num = 0
characters = []
label1,label2,label3,label4,label5,label6 = [],[],[],[],[],[]
for index in range(len(train['content'])):
    if pd.isna(train['emotions'][index]) == False:
        content.append(train['content'][index])
        emotions.append(train['emotions'][index])
        characters.append(train['character'][index])
        current_emotion = train['emotions'][index].split(',')
        label1.append(int(current_emotion[0]))
        label2.append(int(current_emotion[1]))
        label3.append(int(current_emotion[2]))
        label4.append(int(current_emotion[3]))
        label5.append(int(current_emotion[4]))
        label6.append(int(current_emotion[5]))

In [8]:
set1 = set(label1)
dict1,dict2,dict3,dict4,dict5,dict6 = {},{},{},{},{},{}
for item in set1:
    dict1.update({item:label1.count(item)})
set2 = set(label2)
for item in set2:
    dict2.update({item:label2.count(item)})
set3 = set(label3)
for item in set3:
    dict3.update({item:label3.count(item)})
set4 = set(label4)
for item in set4:
    dict4.update({item:label4.count(item)})
set5 = set(label5)
for item in set5:
    dict5.update({item:label5.count(item)})
set6 = set(label6)
for item in set6:
    dict6.update({item:label6.count(item)})

In [9]:
import pandas as pd
import itertools
import json
import math
from bertmodels import Config
from nezha import Bert
base_dir = r'nezha-base'

vocab_file = base_dir + r'/vocab.txt'
vocab_size = len(open(vocab_file,'r').readlines()) 
with open(base_dir+ '/config.json','r',encoding='utf8')as fp:
    json_data = json.load(fp)
json_data['vocab_size'] = vocab_size
config = Config(**json_data)

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from tokenization import FullTokenizer
import numpy as np
tokenizer = FullTokenizer(vocab_file=vocab_file)
from tqdm import tqdm
config.with_mlm = False
#config.with_pooler = True
bertmodel = Bert(config)

---__init__ Nezha


In [10]:
from loader_nezha import load_bert_data
bertmodel = load_bert_data(bertmodel,base_dir + '/pytorch_model.bin')

!!!load Pytorch model!!!
Done loading 196 NEZHA weights from: nezha-base/pytorch_model.bin. Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert.pooler.dense.weight
	cls.predictions.decoder.weight
	cls.predictions.transform.dense.weight
	bert.pooler.dense.bias
	cls.predictions.bias
	cls.predictions.transform.LayerNorm.bias
	cls.predictions.transform.LayerNorm.weight
	cls.seq_relationship.bias
	cls.predictions.transform.dense.bias
	cls.seq_relationship.weight


In [11]:
class ClassificationDataset(Dataset):
    def __init__(self,text,characters,label1,label2,label3,label4,label5,label6,maxlen,flag):
        self.text = text
        self.maxlen = maxlen
        self.characters = characters
        #self.label = [label1,label2,label3,label4,label5,label6]
        token_data,token_id,segment_id,mask_id = [],[],[],[]
        #sequence填充可以最后统一实现
        for index in tqdm(range(len(self.text))):
            current_text = text[index]
            current_character = characters[index]
            current_token = tokenizer.tokenize(current_text)
            current_str_index = inv_new_content_dict[current_text]

            current_key_index = dict_keys_index[current_str_index]
            while current_key_index != -1 and new_content_dict[keys[current_key_index]] == current_text:
                current_key_index = current_key_index-1
            #找寻前面语句的过程之中去重
            if current_key_index == -1:
                pre_content = '无'
            else:
                pre_content = new_content_dict[keys[current_key_index]]
            #找到前面去重之后的语句内容
            new_str_index = keys[current_key_index]
            current_str_index = current_str_index.split('_')
            new_str_index = new_str_index.split('_')
            if current_str_index[0] != new_str_index[0]:
                pre_content = '无'
            if current_str_index[1] != new_str_index[1]:
                pre_content = '无'
            if str(current_character) == 'nan':
                current_character = '无'
            current_character = tokenizer.tokenize(current_character)
            
            #不属于同一个剧本杀的剧本
            pre_token = tokenizer.tokenize(pre_content)
            current_token = ["[CLS]"]+pre_token+["[SEP]"]+current_token+["[SEP]"]+ current_character +["[SEP]"]
#             if str(current_character) != 'nan' and pre_content != '':
#                 pre_token = tokenizer.tokenize(pre_content)
#                 current_token = ["[CLS]"]+pre_token+["[SEP]"]+current_token+["[SEP]"]+[current_character]+["[SEP]"]
#             elif str(current_character) != 'nan' and pre_content == '':
#                 current_token = ["[CLS]"]+current_token+["[SEP]"]+[current_character]+["[SEP]"]
#             elif str(current_character) == 'nan' and pre_content != '':
#                 pre_token = tokenizer.tokenize(pre_content)
#                 current_token = ["[CLS]"]+pre_token+["[SEP]"]+current_token+["[SEP]"]
#             elif str(current_character) == 'nan' and pre_content == '':
#                 current_token = ["[CLS]"]+current_token+["[SEP]"]
                
            r"""
            if pre_content == '':
                pre_content = '无上文'
            if str(current_character) == 'nan':
                current_character = '无角色'
            pre_token = tokenizer.tokenize(pre_content)
            #current_token = ["[CLS]"]+pre_token+["[SEP]"]+current_token+["[SEP]"]+[current_character]+["[SEP]"]
            current_token = ["[CLS]"]+current_token+["[SEP]"]+pre_token+["[SEP]"]+[current_character]+["[SEP]"]
            """
            current_id = tokenizer.convert_tokens_to_ids(current_token)            
            current_id = self.sequence_padding(current_id)
            token_data.append(current_token)
            token_id.append(current_id)
        self.token_data = token_data
        self.token_id = token_id
        self.label = [label1,label2,label3,label4,label5,label6]
        self.tensors = [torch.tensor(self.token_id,dtype=torch.long),
                 torch.tensor(self.label[0],dtype=torch.long),
                 torch.tensor(self.label[1],dtype=torch.long),
                 torch.tensor(self.label[2],dtype=torch.long),
                 torch.tensor(self.label[3],dtype=torch.long),
                 torch.tensor(self.label[4],dtype=torch.long),
                 torch.tensor(self.label[5],dtype=torch.long)]
        
    def __len__(self):
        return len(self.token_id)
    
    def __getitem__(self,index):
        return tuple(tensor[index] for tensor in self.tensors)

    def sequence_padding(self,inputs,padding = 0):
        length = self.maxlen
        if len(inputs) > length:
            inputs = inputs[:length]
        outputs = []
        pad_width = (0,length-len(inputs))
        x = np.pad(inputs,pad_width,'constant',constant_values=padding)
        return x

In [12]:
text = content
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import numpy as np
ss = StratifiedKFold(n_splits=5,shuffle=True,random_state=2)
#建立4折交叉验证方法 查一下KFold函数的参数
text = np.array(text)
label1 = np.array(label1)
label2 = np.array(label2)
label3 = np.array(label3)
label4 = np.array(label4)
label5 = np.array(label5)
label6 = np.array(label6)
characters = np.array(characters)
for train_index,test_index in ss.split(text,label1):
    train_text = text[np.array(train_index)]
    test_text = text[test_index]
    train_characters = characters[train_index]
    test_characters = characters[test_index]
    train_label1 = label1[train_index]
    test_label1 = label1[test_index]
    train_label2 = label2[train_index]
    test_label2 = label2[test_index]
    train_label3 = label3[train_index]
    test_label3 = label3[test_index]
    train_label4 = label4[train_index]
    test_label4 = label4[test_index]
    train_label5 = label5[train_index]
    test_label5 = label5[test_index]
    train_label6 = label6[train_index]
    test_label6 = label6[test_index]
train_dataset = ClassificationDataset(train_text,train_characters,train_label1,train_label2,train_label3,train_label4,train_label5,train_label6,maxlen=350,flag=True)
test_dataset = ClassificationDataset(test_text,test_characters,test_label1,test_label2,test_label3,test_label4,test_label5,test_label6,maxlen=350,flag=False)
#到里面的classificationdataset才进行字符的切割以及划分
train_loader = DataLoader(train_dataset,batch_size=16,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=16)

100%|██████████| 7356/7356 [00:02<00:00, 3196.17it/s]


In [13]:
class ClassificationModel(nn.Module):
    def __init__(self, model, config, n_classes):
        super(ClassificationModel,self).__init__()
        #self.embedding = nn.Embedding(30522,768)
        self.model = model
        self.fc1 = nn.Linear(config.embedding_size,128)
        self.activation = F.relu
        self.dropout = nn.Dropout(0.2)
        #self.activation = F.tanh
        self.out = nn.Linear(128, 6)
        self.sorrow = nn.Linear(6, 1)
        
    def forward(self,input_ids,segment_ids,input_mask):
        #outputs = self.embedding(input_ids)
        output = self.model(input_ids)
        #[64,128,768]
        output = self.dropout(output)
        output = output[:,0]
        output = self.fc1(output)
        output = self.activation(output)
        logits = self.dropout(output)
        pooled_output = self.out(logits)
        tmp_out = torch.transpose(pooled_output, 0, 1)
        love = tmp_out[0]
        joy = tmp_out[1]
        fright = tmp_out[2]
        anger = tmp_out[3]
        fear = tmp_out[4]
        sorrow = tmp_out[5]

#         sorrow = self.sorrow(torch.cat((love.unsqueeze(-1), joy.unsqueeze(-1), fright.unsqueeze(-1), anger.unsqueeze(-1),
#                                         fear.unsqueeze(-1), sorrow.unsqueeze(-1)), dim=-1))
        return {
            'love': love, 'joy': joy, 'fright': fright,
            'anger': anger, 'fear': fear, 'sorrow': sorrow,
        }

In [14]:
from tqdm import tqdm
model = ClassificationModel(bertmodel,config,1)

In [17]:
import torch.nn.functional as F
from torch.optim.lr_scheduler import LambdaLR
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-6)
#不动态调整学习率的时候，lr=2e-5的时候最好值0.420293
def lr_lambda(epoch):
    if epoch > 5:
        return 1
    else:
        return 2/(epoch+1)
scheduler = LambdaLR(optimizer, lr_lambda)
print("初始化的学习率：", optimizer.defaults['lr'])
bestpoint = 0.0
for epoch in range(10):
    print('epoch {}'.format(epoch))
    train_loss = 0
    train_acc = 0
    
    model.train()
    
    model = model.to(device)
    model = nn.DataParallel(model)
    loss_fn = torch.nn.MSELoss(reduce=True,size_average=True)

    losses_love = []
    losses_joy = []
    losses_fright = []
    losses_anger = []
    losses_fear = []
    losses_sorrow = []

    for batch_token_ids,batch_label1,batch_label2,batch_label3,batch_label4,batch_label5,batch_label6 in tqdm(train_loader):
        torch.set_printoptions(edgeitems=768)# 设置输出矩阵维度为768
        #print('batch_token_ids')
        #print(batch_token_ids)
        batch_token_ids = batch_token_ids.to(device)
        batch_label1 = torch.tensor([batch_label1.numpy()], dtype=torch.float)
        batch_label2 = torch.tensor([batch_label2.numpy()], dtype=torch.float)
        batch_label3 = torch.tensor([batch_label3.numpy()], dtype=torch.float)
        batch_label4 = torch.tensor([batch_label4.numpy()], dtype=torch.float)
        batch_label5 = torch.tensor([batch_label5.numpy()], dtype=torch.float)
        batch_label6 = torch.tensor([batch_label6.numpy()], dtype=torch.float)
        logits = model(batch_token_ids,None,None)
        optimizer.zero_grad()
        pred1 = logits['love'].unsqueeze(-1)
        pred2 = logits['joy'].unsqueeze(-1)
        pred3 = logits['fright'].unsqueeze(-1)
        pred4 = logits['anger'].unsqueeze(-1)
        pred5 = logits['fear'].unsqueeze(-1)
        pred6 = logits['sorrow'].unsqueeze(-1)


        loss_love = loss_fn(torch.transpose(pred1, 0, 1), batch_label1.to(device))
        loss_joy = loss_fn(torch.transpose(pred2, 0, 1), batch_label2.to(device))
        loss_fright = loss_fn(torch.transpose(pred3, 0, 1), batch_label3.to(device))
        loss_anger = loss_fn(torch.transpose(pred4, 0, 1), batch_label4.to(device))
        loss_fear = loss_fn(torch.transpose(pred5, 0, 1), batch_label5.to(device))
        loss_sorrow = loss_fn(torch.transpose(pred6, 0, 1), batch_label6.to(device))

        #for index in range(len(batch_labels)):
        #    batch_labels[index] = batch_labels[index].to(device)
        loss = loss_love + loss_joy + loss_fright + loss_anger + loss_fear + loss_sorrow
        losses_love.append(loss_love.item())
        losses_joy.append(loss_joy.item())
        losses_fright.append(loss_fright.item())
        losses_anger.append(loss_anger.item())
        losses_fear.append(loss_fear.item())
        losses_sorrow.append(loss_sorrow.item())
        train_loss = train_loss + loss / 6.0
        loss.backward()
        optimizer.step()
    
    scheduler.step()
    print("第%d个epoch的学习率：%f" % (epoch, optimizer.param_groups[0]['lr']))
    #注意必须从epoch=1开始，否则第0个没有学习率
    print('Train Loss: {:.6f}'.format(train_loss))
    print('love [%s] joy [%s] fright[%s] anger[%s] fear[%s] sorrow[%s]' % (np.mean(losses_love), np.mean(losses_joy), np.mean(losses_fright), np.mean(losses_anger),
                                                                          np.mean(losses_fear), np.mean(losses_sorrow)))
    
    model = model.to(device)
    model.eval()
    
    eval_true_label = [[],[],[],[],[],[]]
    eval_predict_label = [[],[],[],[],[],[]]
    
    #for batch_token_ids,batch_labels in tqdm(test_loader,bar_format='{l_bar}%s{bar}%s{r_bar}' % (Fore.BLUE, Fore.RESET)):
    for batch_token_ids,batch_label1,batch_label2,batch_label3,batch_label4,batch_label5,batch_label6 in tqdm(test_loader):
        batch_token_ids = batch_token_ids.to(device)
        batch_labels = [batch_label1,batch_label2,batch_label3,batch_label4,batch_label5,batch_label6]
        with torch.no_grad():
            output = model(batch_token_ids,None,None)
        lst = ['love', 'joy', 'fright', 'anger', 'fear', 'sorrow']
        for index in lst:
            for index1 in range(len(output[index])):
                
                abs0 = abs(output[index][index1]-0)
                abs1 = abs(output[index][index1]-1)
                abs2 = abs(output[index][index1]-2)
                abs3 = abs(output[index][index1]-3)
                currentdata = [abs0,abs1,abs2,abs3]
                current_label = currentdata.index(min(currentdata))
                #eval_predict_label[index1].append(current_label)
                current_predict = output[index][index1].item()
                #当前类别的分类结果,这里append(output[index][index1])
                #直接放入对应概率值时效果最好,这里直接放入0,1,2,3对应的数值
                eval_predict_label[lst.index(index)].append(current_predict)
        for index in range(6):
            if index == 0:
                current_label = np.array(batch_label1.cpu()).tolist()
            if index == 1:
                current_label = np.array(batch_label2.cpu()).tolist()
            if index == 2:
                current_label = np.array(batch_label3.cpu()).tolist()
            if index == 3:
                current_label = np.array(batch_label4.cpu()).tolist()
            if index == 4:
                current_label = np.array(batch_label5.cpu()).tolist()
            if index == 5:
                current_label = np.array(batch_label6.cpu()).tolist()
            eval_true_label[index].extend(current_label)
    criterion = nn.MSELoss()
    totalloss = 0

    for index in range(len(eval_true_label)):
#         if index == 5:
#             for j in range(len(eval_predict_label[index])):
#                 eval_predict_label[index][j] *= 0.67
        inputs = torch.autograd.Variable(torch.from_numpy(np.array(eval_predict_label[index])))
        target = torch.autograd.Variable(torch.from_numpy(np.array(eval_true_label[index])))
        current_loss = criterion(inputs.float(),target.float())
        current_loss = current_loss.item()
        print('index = %d,current_loss = %f'% (index,current_loss))
        totalloss = totalloss + current_loss
    
    #totalloss = totalloss/len(eval_predict_label)
    print('totalloss = ')
    print(totalloss)
    totalloss = totalloss/6
    totalloss = math.sqrt(totalloss)
    currentpoint = 1/(1+totalloss)
    #currentpoint = 1/(1+current_loss)
    print('current_point = ')
    print(currentpoint)
    if currentpoint > bestpoint:
        bestpoint = currentpoint
        torch.save(model,'best_score'+str(bestpoint)+'.pth')

初始化的学习率： 5e-06
epoch 0


100%|██████████| 1840/1840 [11:26<00:00,  2.68it/s]


第0个epoch的学习率：0.000005
Train Loss: 373.943848
love [0.13889859567624743] joy [0.12266608508747912] fright[0.13859603939123663] anger[0.23516315863298193] fear[0.21413416567129784] sorrow[0.36992442339668863]


100%|██████████| 460/460 [01:06<00:00,  6.89it/s]


index = 0,current_loss = 0.135675
index = 1,current_loss = 0.130319
index = 2,current_loss = 0.132856
index = 3,current_loss = 0.214879
index = 4,current_loss = 0.188604
index = 5,current_loss = 0.339719
totalloss = 
1.1420523524284363
current_point = 
0.696242061088765
epoch 1


...

index = 0,current_loss = 0.133456
index = 1,current_loss = 0.116998
index = 2,current_loss = 0.129051
index = 3,current_loss = 0.193720
index = 4,current_loss = 0.199007
index = 5,current_loss = 0.351470
totalloss = 
1.12370203435421
current_point = 
0.6979522184250958
epoch 9


100%|██████████| 1840/1840 [11:40<00:00,  2.63it/s]


第9个epoch的学习率：0.000005
Train Loss: 153.911484
love [0.05923651377131097] joy [0.06193704367460273] fright[0.06883532854367133] anger[0.08352352738451294] fear[0.09482368895098815] sorrow[0.133529397457175]


100%|██████████| 460/460 [01:10<00:00,  6.57it/s]

index = 0,current_loss = 0.128778
index = 1,current_loss = 0.112035
index = 2,current_loss = 0.128064
index = 3,current_loss = 0.181980
index = 4,current_loss = 0.175611
index = 5,current_loss = 0.346924
totalloss = 
1.073391169309616
current_point = 
0.702758477507092


In [33]:
## model = torch.load('best_score'+str(bestpoint)+'.pth')
test=pd.read_csv('data/test_dataset.tsv', sep='\t')

In [34]:
model = torch.load('best_score0.7111817192757776.pth')

In [35]:
content_dict = {}
for index in range(len(test['content'])):
    currentindex = test['id'][index]
    currentindex = currentindex.split('_')
    str1 = currentindex[0]
    while len(str1) < 5:
        str1 = '0'+str1
    str4 = currentindex[3]
    while len(str4) < 4:
        str4 = '0'+str4
    resultindex = str1+'_'+currentindex[1]+'_'+currentindex[2]+'_'+str4
    content_dict[resultindex] = test['content'][index]

In [36]:
def sortedDictValues(adict): 
    keys = adict.keys() 
    keys = sorted(keys)
    print('keys = ')
    print(keys[0:10])
    new_content_dict = {}
    for data in keys:
        new_content_dict[data] = content_dict[data]
    return new_content_dict 
new_content_dict = sortedDictValues(content_dict)

keys = 
['01597_0001_A_0001', '01597_0001_A_0002', '01597_0001_A_0003', '01597_0001_A_0004', '01597_0001_A_0005', '01597_0001_A_0006', '01597_0001_A_0007', '01597_0001_A_0008', '01597_0001_A_0009', '01597_0001_A_0010']


In [37]:
inv_new_content_dict = {}
for data in new_content_dict:
    inv_new_content_dict[new_content_dict[data]] = data

In [38]:
keys = new_content_dict.keys()
keys = list(keys)
dict_keys_index = {}
for index in range(len(keys)):
    #dict_keys_index[index] = keys[index]
    dict_keys_index[keys[index]] = index

In [39]:
content,emotions = [],[]
dicts = {}
inv_dicts = {}
label_num = 0
characters = []
for index in range(len(test['content'])):
        content.append(test['content'][index])
        characters.append(test['character'][index])

In [40]:
testtext = test['content']
testid = test['id']
testcharacter = test['character']
class TestDataset(Dataset):
    def __init__(self,text,character,maxlen):
        self.text = text
        self.maxlen = maxlen
        token_data,token_id,segment_id,mask_id = [],[],[],[]
        #sequence填充可以最后统一实现
        for index in tqdm(range(len(self.text))):
            current_text = text[index]
            current_character = characters[index]
            current_token = tokenizer.tokenize(current_text)
            current_str_index = inv_new_content_dict[current_text]

            current_key_index = dict_keys_index[current_str_index]
            while current_key_index != -1 and new_content_dict[keys[current_key_index]] == current_text:
                current_key_index = current_key_index-1
            #找寻前面语句的过程之中去重
            if current_key_index == -1:
                pre_content = ''
            else:
                pre_content = new_content_dict[keys[current_key_index]]
            
            new_str_index = keys[current_key_index]
            current_str_index = current_str_index.split('_')
            new_str_index = new_str_index.split('_')
            if current_str_index[0] != new_str_index[0]:
                pre_content = '无'
            if current_str_index[1] != new_str_index[1]:
                pre_content = '无'
            if str(current_character) == 'nan':
                current_character = '无'
            pre_token = tokenizer.tokenize(current_character)
            #不属于同一个剧本杀的剧本
            pre_token = tokenizer.tokenize(pre_content)
            current_token = ["[CLS]"]+pre_token+["[SEP]"]+current_token+["[SEP]"]+[current_character]+["[SEP]"]
            
#             if str(current_character) != 'nan' and pre_content != '':
#                 pre_token = tokenizer.tokenize(pre_content)
#                 current_token = ["[CLS]"]+pre_token+["[SEP]"]+current_token+["[SEP]"]+[current_character]+["[SEP]"]
#             elif str(current_character) != 'nan' and pre_content == '':
#                 current_token = ["[CLS]"]+current_token+["[SEP]"]+[current_character]+["[SEP]"]
#             elif str(current_character) == 'nan' and pre_content != '':
#                 pre_token = tokenizer.tokenize(pre_content)
#                 current_token = ["[CLS]"]+pre_token+["[SEP]"]+current_token+["[SEP]"]
#             elif str(current_character) == 'nan' and pre_content == '':
#                 current_token = ["[CLS]"]+current_token+["[SEP]"]
            
            r"""
            if pre_content == '':
                pre_content = '无上文'
            if str(current_character) == 'nan':
                current_character = '无角色'
            pre_token = tokenizer.tokenize(pre_content)
            #current_token = ["[CLS]"]+pre_token+["[SEP]"]+current_token+["[SEP]"]+[current_character]+["[SEP]"]
            current_token = ["[CLS]"]+current_token+["[SEP]"]+pre_token+["[SEP]"]+[current_character]+["[SEP]"]
            """
            current_id = tokenizer.convert_tokens_to_ids(current_token)            
            current_id = self.sequence_padding(current_id)
            token_data.append(current_token)
            token_id.append(current_id)
        self.token_data = token_data
        self.token_id = token_id
        #self.segment_id = sequence_padding(self.segment_id,maxlen)
        #self.mask_id = sequence_padding(self.mask_id,maxlen)
        self.tensors = [torch.tensor(self.token_id,dtype=torch.long)]
        
    def __len__(self):
        return len(self.token_id)
    
    def __getitem__(self,index):
        return tuple(tensor[index] for tensor in self.tensors)
    
    def sequence_padding(self,inputs,padding = 0):
        length = self.maxlen
        if len(inputs) > length:
            inputs = inputs[:length]
        outputs = []
        pad_width = (0,length-len(inputs))
        x = np.pad(inputs,pad_width,'constant',constant_values=padding)
        return x

In [29]:
test_dataset = TestDataset(testtext,testcharacter,maxlen=200)
test_loader = DataLoader(test_dataset,batch_size=10,shuffle=False)

100%|██████████| 21376/21376 [00:06<00:00, 3182.85it/s]


## 修改测试集的评分标准

In [42]:
model = model.to(device)
model.eval()
eval_loss = 0.
eval_acc = 0.
eval_predict_label = []
index = []
pred = [[],[],[],[],[],[]]
current_index = 0
for batch_token_ids in tqdm(test_loader):
    batch_token_ids = batch_token_ids[0].to(device)
    with torch.no_grad():
        output = model(batch_token_ids,None,None)
        lst = ['love', 'joy', 'fright', 'anger', 'fear', 'sorrow']

        for index in lst:
            for index1 in range(len(output[index])):
                #对应的类别概率0~6
                abs0 = abs(output[index][index1]-0)
                abs1 = abs(output[index][index1]-1)
                abs2 = abs(output[index][index1]-2)
                abs3 = abs(output[index][index1]-3)
                currentdata = [abs0,abs1,abs2,abs3]
                current_label = currentdata.index(min(currentdata))
                #eval_predict_label[index1].append(current_label)
                current_predict = output[index][index1].item()
                pred[lst.index(index)].append(current_predict)
                #eval_predict_label[index1].append(current_predict)
                #当前类别的分类结果,这里append(output[index][index1])
                #直接放入对应概率值时效果最好,这里直接放入0,1,2,3对应的数值
for index in range(len(pred[0])):
    for i in range(6):
        if pred[i][index] <= 0.0001:
            pred[i][index] = 0.0
        if pred[i][index] >= 3.0:
            pred[i][index] = 3.0
#         if i == 5:
#             pred[i][index] *= 0.78
    eval_predict_label.append(str(pred[0][index])+','+str(pred[1][index])+','+str(pred[2][index])+','+str(pred[3][index])+','+str(pred[4][index])+','+str(pred[5][index]))
result_data = []
for index in range(len(testid)):
    result_data.append([testid[index],eval_predict_label[index]])
#pd.DataFrame({"id":testid,"label":eval_predict_label}).to_csv("/home/xiaoguzai/代码/剧本角色情感识别/数据集/crossentropy"+str(bestpoint)+"result.csv",index=False)

100%|██████████| 2138/2138 [01:53<00:00, 18.80it/s]


a=np.array([[1,2],[3,4]])
b=np.array([[2,3],[4,5]])
loss_fn = torch.nn.MSELoss(reduce=False, size_average=False)
input = torch.autograd.Variable(torch.from_numpy(a))
target = torch.autograd.Variable(torch.from_numpy(b))
loss = loss_fn(input.float(),target.float())

In [43]:
import csv
with open(f'outputs/nezha_%.4f_result.csv' % bestpoint, 'w') as f:
    tsv_w = csv.writer(f, delimiter='\t')
    tsv_w.writerow(['id', 'label'])  # 单行写入
    tsv_w.writerows(result_data)  # 多行写入